In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
url = "https://datahub.io/core/s-and-p-500-companies/r/constituents.csv"
df = pd.read_csv(url)

# Standardize columns
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

print(df.head())
print(df.info())

  symbol             security             gics_sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                gics_sub-industry    headquarters_location  date_added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       cik      founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 en

In [35]:
# Check missing values
print(df.isnull().sum())

# Drop rows missing essential identifiers
df = df.dropna(subset=["symbol", "security"])

# Fill categorical missing values
categorical_cols = [
    "gics_sector",
    "gics_sub-industry",
    "headquarters_location"
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

symbol                   0
security                 0
gics_sector              0
gics_sub-industry        0
headquarters_location    0
date_added               0
cik                      0
founded                  0
sector_code              0
subindustry_code         0
dtype: int64


In [36]:
# Convert 'date_added' to datetime
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")

# Extract year added
df["year_added"] = df["date_added"].dt.year

# Years in S&P 500
df["years_in_sp500"] = 2025 - df["year_added"]

# Drop original date column
df = df.drop(columns=["date_added"])

In [37]:
# Handle values like "2013 (1888)"
df["founded"] = (
    df["founded"]
    .astype(str)
    .str.extract(r"(\d{4})")
    .astype(float)
)

# Company age
df["company_age"] = 2025 - df["founded"]

In [38]:
# Convert categories to numeric codes
df["sector_code"] = df["gics_sector"].astype("category").cat.codes
df["subindustry_code"] = df["gics_sub-industry"].astype("category").cat.codes

In [39]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

numeric_cols = [
    "cik",
    "company_age",
    "years_in_sp500",
    "sector_code",
    "subindustry_code"
]

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])